Created by Tomer Danon

### Data Cleaning & Preperation

This notebook cleans the data and saves two dataframes as pickle files to save time and for easy loading in beerReviews_Recommendations.ipynb, which is where the EDA and model recommendation algorithms are stored.
- **main.pickle**: this is the main dataframe after some basic cleaning.
- **users.pickle**: this is the user dataframe which is created through a series of data manipulation techniques. It provides a column for each user’s beer and beer style preferences and ratings, as well as the total number of reviews they submitted. The functions associates with creating this dataframe each take about an hour to run; hence it is saved as a pickle file.



NOTE: The pickle files are provided for easy loading into beerReviews_Recommendations.ipynb. There is no need to run this notebook. It is for reference only. 

*** If you do choose to run it, it may take up to two hours to complete.


In [1]:
import pandas as pd
import pickle

In [2]:
# Read data into Pandas dataframes
beer_reviews_1 = pd.read_csv("./beer_reviews_1.csv")
beer_reviews_2 = pd.read_csv("./beer_reviews_2.csv")

In [3]:
# Join the two dataframes
main = pd.concat([beer_reviews_1, beer_reviews_2], axis=0)
# Change review_overall to actually be the average of the other review columns
main['review_overall'] = main[['review_aroma', 'review_appearance', 'review_palate', 'review_taste']].mean(axis=1)
main.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.875,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,2.875,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,2.875,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.000,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.250,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [4]:
# Inspect NaNs
main.isna().sum()

brewery_id                0
brewery_name             15
review_time               0
review_overall            0
review_aroma              0
review_appearance         0
review_profilename      348
beer_style                0
review_palate             0
review_taste              0
beer_name                 0
beer_abv              67785
beer_beerid               0
dtype: int64

In [5]:
# Remove rows with NaN. Remove review time - it is unknown what this attribute refers to or what format it is in.
print(main.shape)
main = main[ main['brewery_name'].notna() ]
main = main[ main['review_profilename'].notna() ] 
main = main.drop('review_time', axis=1)
print(main.shape)

(1586614, 13)
(1586251, 12)


In [6]:
# Inspect NaNs again
main.isna().sum()

brewery_id                0
brewery_name              0
review_overall            0
review_aroma              0
review_appearance         0
review_profilename        0
beer_style                0
review_palate             0
review_taste              0
beer_name                 0
beer_abv              67773
beer_beerid               0
dtype: int64

In [7]:
# Save main cleaned main dataframe to pickle file for easy loading
main.to_pickle('./main.pickle')

In [7]:
# Inspect DataFrame info
main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1586251 entries, 0 to 714712
Data columns (total 12 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   brewery_id          1586251 non-null  int64  
 1   brewery_name        1586251 non-null  object 
 2   review_overall      1586251 non-null  float64
 3   review_aroma        1586251 non-null  float64
 4   review_appearance   1586251 non-null  float64
 5   review_profilename  1586251 non-null  object 
 6   beer_style          1586251 non-null  object 
 7   review_palate       1586251 non-null  float64
 8   review_taste        1586251 non-null  float64
 9   beer_name           1586251 non-null  object 
 10  beer_abv            1518478 non-null  float64
 11  beer_beerid         1586251 non-null  int64  
dtypes: float64(6), int64(2), object(4)
memory usage: 157.3+ MB


In [8]:
# Description of numerical attributes
main.loc[ : , 'review_overall':'beer_abv'].describe()

,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv
count,1.586251e+06,1.586251e+06,1.586251e+06,1.586251e+06,1.586251e+06,1.518478e+06
mean,3.778512e+00,3.735688e+00,3.841677e+00,3.743757e+00,3.792925e+00,7.042488e+00
std,5.793851e-01,6.975649e-01,6.160621e-01,6.821700e-01,7.319083e-01,2.322568e+00
min,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02
25%,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00
50%,3.875000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00
75%,4.125000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00
max,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01


### Users

In [9]:
# Create a DataFrame of unique users
users = pd.DataFrame(main.groupby('review_profilename').count()['brewery_id']).rename(columns={'brewery_id':'num_reviews'})
users

,num_reviews
review_profilename,
0110x011,139
01Ryan10,1
02maxima,4
03SVTCobra,3
04101Brewer,3
...,...
zymurgy4all,94
zymurgywhiz,2
zythus,8


In [10]:
def user_styles(review_profilename):
    '''
    Function to create a dictionary of beer styles a user has reviewed.
    Beer style as the keys, rating as the values.
    '''
    group = main[ main.review_profilename == review_profilename].groupby('beer_style')
    df = pd.DataFrame(group.mean()['review_overall'].round(2))
    df['num_reviews'] = group.count()['review_overall'].values
    styles = {}
    for i in zip(df.index, df.review_overall):
        styles[i[0]] = i[1]
    return styles

users['user_styles'] = users.index.copy()
users['user_styles'] = users['user_styles'].apply(user_styles)
users    

,num_reviews,user_styles
review_profilename,,
0110x011,139,"{'American Amber / Red Ale': 3.88, 'American B..."
01Ryan10,1,{'American IPA': 4.88}
02maxima,4,"{'American Double / Imperial IPA': 4.62, 'Belg..."
03SVTCobra,3,"{'English Porter': 3.5, 'Light Lager': 1.75, '..."
04101Brewer,3,"{'American IPA': 3.75, 'American Pale Ale (APA..."
...,...,...
zymurgy4all,94,"{'American Adjunct Lager': 3.25, 'American Amb..."
zymurgywhiz,2,"{'American Amber / Red Ale': 3.88, 'American I..."
zythus,8,"{'Bock': 4.12, 'Dubbel': 4.38, 'English Brown ..."


In [11]:
# Save to pickle for easy loading
users.to_pickle('./users.pickle')

In [14]:
def user_beers(review_profilename):
    '''
    Function to create a dictionary of beers a user has reviewed.
    Beer name as the keys, beer ID, brewery name, beer style and score as the values.
    '''
    df = pd.DataFrame(main[ main.review_profilename == review_profilename].groupby(['beer_beerid', 'beer_name', 'brewery_name', 'beer_style']).mean()['review_overall'].round(2))
    beers = {}
    for i in zip(df.index, df.review_overall):
        beers[i[0][1]] = (i[0][0], i[0][2], i[0][3], i[1])
    return beers

users['user_beers'] = users.index.copy()
users['user_beers'] = users['user_beers'].apply(user_beers)
users   

,num_reviews,user_styles,user_beers
review_profilename,,,
0110x011,139,"{'American Amber / Red Ale': 3.88, 'American B...","{'Wachusett IPA (India Pale Ale)': (23, 'Wachu..."
01Ryan10,1,{'American IPA': 4.88},"{'Duet IPA': (27604, 'Alpine Beer Company', 'A..."
02maxima,4,"{'American Double / Imperial IPA': 4.62, 'Belg...","{'Brooklyn Black Chocolate Stout': (680, 'Broo..."
03SVTCobra,3,"{'English Porter': 3.5, 'Light Lager': 1.75, '...","{'Lone Star Light': (1731, 'Pabst Brewing Comp..."
04101Brewer,3,"{'American IPA': 3.75, 'American Pale Ale (APA...","{'Peeper Ale': (51512, 'Maine Beer Company', '..."
...,...,...,...
zymurgy4all,94,"{'American Adjunct Lager': 3.25, 'American Amb...","{'Mackeson Triple XXX Stout': (19, 'Whitbread ..."
zymurgywhiz,2,"{'American Amber / Red Ale': 3.88, 'American I...","{'Sierra Nevada Celebration Ale': (1904, 'Sier..."
zythus,8,"{'Bock': 4.12, 'Dubbel': 4.38, 'English Brown ...","{'Samuel Smith's, The Famous Taddy Porter': (5..."


In [15]:
# Save to pickle for easy load into recommendation file
users.to_pickle('./users.pickle')